# Day 29: 最終プロジェクト2 - 画像分類アプリケーション

## Learning Objectives
- 完全な画像分類パイプラインを構築する
- 前処理から評価までの一連の流れを実装する
- GUIインターフェースを追加する
- モデルのデプロイと実運用をシミュレートする

---

# Part 1: Project Overview

## 29.1 プロジェクト概要

### アプリケーションの仕様

**目的**: 画像分類を行う完全なWebアプリケーション
**対象画像**: 手書き数字、カテゴリ分類可能な画像
**機能**:
1. **画像アップロード**: ユーザーが画像をアップロード
2. **前処理**: 画像サイズの統一、正規化
3. **特徴量抽出**: 画像から特徴ベクトルを生成
4. **分類**: 学習済みモデルで分類を実行
5. **結果表示**: 分類結果と信頼度を表示
6. **履歴機能**: 過去の分類結果を保存

**技術スタック**:
- **バックエンド**: Python（Flask）
- **フロントエンド**: HTML, CSS, JavaScript
- **機械学習**: scikit-learn（実装済みのk-NN）
- **画像処理**: OpenCV（外部ライブラリ許容）

## 29.2 アーキテクチャ設計

### システム構成

```
┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐
│   Webブラウザ    │    │   Flaskサーバー  │    │   学習済みモデル  │
│                 │    │                 │    │                 │
│ - 画像アップロード│───▶│ - リクエスト処理 │───▶│ - 特徴抽出      │
│ - 結果表示      │    │ - 画像前処理     │    │ - 分類推論      │
│ - 履歴表示      │    │ - JSONレスポンス│    │                 │
│                 │    │                 │    └─────────────────┘
└─────────────────┘    └─────────────────┘
        │                       │
        │                       ▼
        └──────────────┐ ┌─────────────────┐
                      │ │   データベース    │
                      │ │                 │
                      │ │ - 分類履歴      │
                      │ │ - ユーザーデータ │
                      │ │                 │
                      └ └─────────────────┘
```

### 主要コンポーネント

1. **ImageProcessor**: 画像の読み込み、前処理
2. **FeatureExtractor**: 特徴量抽出
3. **Classifier**: 分類器（k-NNベース）
4. **WebServer**: Flaskサーバー
5. **Database**: SQLiteで履歴保存
6. **WebUI**: ユーザーインターフェース

---

# Part 2: Implementation

In [ ]:
# 必要なライブラリのインポート
import os
import numpy as np
import cv2
from PIL import Image
import json
import sqlite3
from datetime import datetime
import base64
from flask import Flask, request, render_template, jsonify, redirect, url_for
import matplotlib.pyplot as plt
from io import BytesIO
import tempfile
import shutil
from collections import deque
import threading
import time

# 学習済みモデルのクラスをインポート（Day23で作成）
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Flaskアプリケーションの初期化
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'static/uploads'
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16MB max upload

# アップロードディレクトリの作成
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)
os.makedirs('static/history', exist_ok=True)
os.makedirs('static/models', exist_ok=True)

# グローバル変数
model = None
scaler = None
feature_extractor = None
class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
history = deque(maxlen=50)  # 最近の50件を保存

## 29.3 画像処理パイプラインの実装

In [1]:
class ImageProcessor:
    """画像処理パイプライン"""
    
    def __init__(self, target_size=(64, 64)):
        self.target_size = target_size
        
    def load_image(self, image_path):
        """画像を読み込む"""
        # OpenCVとPILの両方で読み込めるように
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            try:
                img = Image.open(image_path)
                img = img.convert('L')  # グレースケールに変換
                img = np.array(img)
            except Exception as e:
                raise ValueError(f"画像の読み込みに失敗しました: {e}")
        return img
    
    def preprocess(self, image):
        """画像の前処理"""
        # 1. リサイズ
        if len(image.shape) == 2:
            resized = cv2.resize(image, self.target_size)
        else:
            # カラー画像の場合はグレースケールに変換
            resized = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(resized, self.target_size)
        
        # 2. 二値化（必要に応じて）
        _, binary = cv2.threshold(resized, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        # 3. ノイズ除去（メディアンフィルタ）
        denoised = cv2.medianBlur(binary, 3)
        
        # 4. 中心化（余白を削除）
        contours, _ = cv2.findContours(denoised, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            x, y, w, h = cv2.boundingRect(contours[0])
            cropped = denoised[y:y+h, x:x+w]
            # 元のサイズにリサイズ
            final = cv2.resize(cropped, self.target_size)
        else:
            final = denoised
        
        return final
    
    def extract_features_hog(self, image):
        """HOG特徴量の抽出"""
        # OpenCVのHOGディスクリプタを使用
        win_size = (image.shape[1], image.shape[0])
        block_size = (16, 16)
        block_stride = (8, 8)
        cell_size = (8, 8)
        nbins = 9
        
        hog = cv2.HOGDescriptor(win_size, block_size, block_stride, 
                                cell_size, nbins)
        
        # 特徴量の計算
        hog_features = hog.compute(image)
        hog_features = hog_features.flatten()
        
        return hog_features
    
    def extract_features_histogram(self, image, bins=16):
        """ヒストグラム特徴量の抽出"""
        # 全体のヒストグラム
        hist = cv2.calcHist([image], [0], None, [bins], [0, 256])
        hist = hist.flatten() / hist.sum()  # 正規化
        
        # 水平方向のヒストグラム
        h_hist = np.mean(image, axis=0)
        h_hist = np.histogram(h_hist, bins=bins, range=(0, 256))[0]
        h_hist = h_hist / h_hist.sum() if h_hist.sum() > 0 else h_hist
        
        # 垂直方向のヒストグラム
        v_hist = np.mean(image, axis=1)
        v_hist = np.histogram(v_hist, bins=bins, range=(0, 256))[0]
        v_hist = v_hist / v_hist.sum() if v_hist.sum() > 0 else v_hist
        
        # 4象限のヒストグラム
        h, w = image.shape
        q1 = image[:h//2, :w//2]
        q2 = image[:h//2, w//2:]
        q3 = image[h//2:, :w//2]
        q4 = image[h//2:, w//2:]
        
        q1_hist = np.histogram(q1.flatten(), bins=bins, range=(0, 256))[0]
        q2_hist = np.histogram(q2.flatten(), bins=bins, range=(0, 256))[0]
        q3_hist = np.histogram(q3.flatten(), bins=bins, range=(0, 256))[0]
        q4_hist = np.histogram(q4.flatten(), bins=bins, range=(0, 256))[0]
        
        # すべての特徴量を結合
        features = np.concatenate([
            hist, h_hist, v_hist,
            q1_hist, q2_hist, q3_hist, q4_hist
        ])
        
        return features
    
    def extract_features_combined(self, image):
        """複数の特徴量を結合"""
        # 基本的な統計量
        features = []
        
        # 1. 平均、分散、標準偏差
        features.extend([
            np.mean(image),
            np.var(image),
            np.std(image)
        ])
        
        # 2. ヒストグラム特徴量
        hist_features = self.extract_features_histogram(image)
        features.extend(hist_features)
        
        # 3. HOG特徴量（サイズが小さい場合はスキップ）
        if image.shape[0] >= 32 and image.shape[1] >= 32:
            try:
                hog_features = self.extract_features_hog(image)
                # HOG特徴量を最初の100次元に制限
                features.extend(hog_features[:100])
            except:
                # HOG計算に失敗した場合はゼロパディング
                features.extend([0] * 100)
        else:
            features.extend([0] * 100)
        
        # 4. 基本的な形状特徴
        contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            cnt = contours[0]
            area = cv2.contourArea(cnt)
            perimeter = cv2.arcLength(cnt, True)
            x, y, w, h = cv2.boundingRect(cnt)
            aspect_ratio = w / h if h > 0 else 0
            extent = area / (w * h) if w * h > 0 else 0
            
            features.extend([area, perimeter, aspect_ratio, extent])
        else:
            features.extend([0] * 4)
        
        return np.array(features)

## 29.4 データベースと履歴管理

In [ ]:
class DatabaseManager:
    """データベース管理クラス"""
    
    def __init__(self, db_path='app.db'):
        self.db_path = db_path
        self.init_database()
    
    def init_database(self):
        """データベースの初期化"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # 履歴テーブル
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS prediction_history (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
            filename TEXT,
            predicted_class TEXT,
            confidence REAL,
            actual_class TEXT,
            processing_time REAL,
            image_data BLOB
        )
        ''')
        
        # ユーザー統計テーブル
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS user_stats (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            date DATE,
            total_predictions INTEGER DEFAULT 0,
            correct_predictions INTEGER DEFAULT 0,
            average_confidence REAL DEFAULT 0.0
        )
        ''')
        
        conn.commit()
        conn.close()
    
    def save_prediction(self, filename, predicted_class, confidence, 
                       actual_class=None, processing_time=None, image_data=None):
        """予測結果を保存"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute('''
        INSERT INTO prediction_history 
        (filename, predicted_class, confidence, actual_class, processing_time, image_data)
        VALUES (?, ?, ?, ?, ?, ?)
        ''', (filename, predicted_class, confidence, actual_class, 
              processing_time, image_data))
        
        conn.commit()
        conn.close()
    
    def get_recent_predictions(self, limit=20):
        """最近の予測履歴を取得"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute('''
        SELECT timestamp, filename, predicted_class, confidence, actual_class
        FROM prediction_history
        ORDER BY timestamp DESC
        LIMIT ?
        ''', (limit,))
        
        results = cursor.fetchall()
        conn.close()
        
        return results
    
    def get_statistics(self):
        """統計情報を取得"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # 全体の統計
        cursor.execute('''
        SELECT 
            COUNT(*) as total,
            SUM(CASE WHEN predicted_class = actual_class THEN 1 ELSE 0 END) as correct,
            AVG(confidence) as avg_confidence,
            AVG(processing_time) as avg_time
        FROM prediction_history
        WHERE actual_class IS NOT NULL
        ''')
        
        stats = cursor.fetchone()
        
        # 各クラスの統計
        cursor.execute('''
        SELECT 
            predicted_class,
            COUNT(*) as count,
            AVG(confidence) as avg_confidence
        FROM prediction_history
        GROUP BY predicted_class
        ORDER BY count DESC
        ''')
        
        class_stats = cursor.fetchall()
        
        conn.close()
        
        return {
            'total': stats[0],
            'correct': stats[1],
            'accuracy': stats[1] / stats[0] if stats[0] > 0 else 0,
            'avg_confidence': stats[2],
            'avg_time': stats[3],
            'class_stats': class_stats
        }
    
    def get_predictions_by_date(self, date):
        """特定日の予測履歴を取得"""
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        cursor.execute('''
        SELECT timestamp, filename, predicted_class, confidence, actual_class
        FROM prediction_history
        WHERE DATE(timestamp) = ?
        ORDER BY timestamp DESC
        ''', (date,))
        
        results = cursor.fetchall()
        conn.close()
        
        return results

# データベースマネージャーの初期化
db = DatabaseManager()

## 29.5 モデルの学習と保存

In [ ]:
def train_model_and_save():
    """モデルを学習して保存"""
    print("モデルの学習を開始...")
    
    # 生成データ（Day23と同じ）
    def generate_digit_images(num_samples=1000, image_size=32):
        images = []
        labels = []
        
        for digit in range(10):
            digit_samples = num_samples // 10
            
            for _ in range(digit_samples):
                image = np.zeros((image_size, image_size), dtype=np.uint8)
                
                # より複雑な数字のパターン
                center = image_size // 2
                thickness = max(2, image_size // 10)
                
                if digit == 0:  # 0
                    cv2.circle(image, (center, center), center-thickness, 200, thickness)
                elif digit == 1:  # 1
                    cv2.line(image, (center, 5), (center, image_size-5), 200, thickness)
                    cv2.line(image, (center-2, center), (center+2, center), 200, thickness)
                elif digit == 2:  # 2
                    pts = np.array([[5, image_size-5], [5, center], [center, 5], 
                                  [image_size-5, center], [image_size-5, image_size-5]], 
                                  np.int32)
                    cv2.polylines(image, [pts], False, 200, thickness)
                elif digit == 3:  # 3
                    cv2.ellipse(image, (center, center), (center-5, center-5), 
                               0, 0, 180, 200, thickness)
                    cv2.ellipse(image, (center, center), (center-5, center-5), 
                               0, 180, 360, 200, thickness)
                elif digit == 4:  # 4
                    cv2.line(image, (5, 5), (5, center), 200, thickness)
                    cv2.line(image, (5, center), (image_size-5, center), 200, thickness)
                    cv2.line(image, (image_size-5, 5), (image_size-5, image_size-5), 200, thickness)
                elif digit == 5:  # 5
                    pts = np.array([[5, 5], [image_size-5, 5], [image_size-5, center], 
                                  [5, center], [5, image_size-5], [image_size-5, image_size-5]], 
                                  np.int32)
                    cv2.polylines(image, [pts], False, 200, thickness)
                elif digit == 6:  # 6
                    cv2.circle(image, (center, center), center-thickness, 200, thickness)
                    cv2.line(image, (center, center), (center, image_size-5), 200, thickness)
                elif digit == 7:  # 7
                    cv2.line(image, (5, 5), (image_size-5, 5), 200, thickness)
                    cv2.line(image, (image_size-5, 5), (center, center), 200, thickness)
                    cv2.line(image, (center, center), (center, image_size-5), 200, thickness)
                elif digit == 8:  # 8
                    cv2.circle(image, (center, center), center-thickness, 200, thickness)
                    cv2.line(image, (5, center), (image_size-5, center), 200, thickness)
                elif digit == 9:  # 9
                    cv2.circle(image, (center, center), center-thickness, 200, thickness)
                    cv2.line(image, (center, center), (center, 5), 200, thickness)
                
                # ランダムなノイズ
                noise = np.random.randint(0, 50, image_size*image_size).reshape(image_size, image_size)
                image = cv2.add(image, noise)
                
                # 輝度調整
                image = cv2.convertScaleAbs(image, alpha=1.2, beta=0)
                
                images.append(image)
                labels.append(digit)
        
        return images, labels
    
    # データの生成
    images, labels = generate_digit_images(num_samples=2000, image_size=32)
    print(f"生成データ: {len(images)}枚")
    
    # 特徴量抽出
    print("特徴量抽出中...")
    processor = ImageProcessor(target_size=(32, 32))
    X = [processor.extract_features_combined(img) for img in images]
    y = labels
    
    # データの分割
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # 特徴量の正規化
    print("特徴量の正規化...")
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # モデルの学習
    print("モデルの学習中...")
    global model
    model = KNeighborsClassifier(n_neighbors=5, weights='distance')
    model.fit(X_train_scaled, y_train)
    
    # モデルの評価
    y_pred = model.predict(X_test_scaled)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"テスト精度: {accuracy:.4f}")
    
    # モデルとスケーラーを保存
    import pickle
    with open('static/models/digit_classifier.pkl', 'wb') as f:
        pickle.dump({'model': model, 'scaler': scaler, 'processor': processor}, f)
    
    print("モデルの保存完了！")
    return accuracy

# 初期モデルの学習（一度だけ）
if model is None:
    try:
        # 保存されたモデルを読み込む
        import pickle
        with open('static/models/digit_classifier.pkl', 'rb') as f:
            saved_data = pickle.load(f)
            model = saved_data['model']
            scaler = saved_data['scaler']
            feature_extractor = saved_data['processor']
        print("保存されたモデルを読み込みました")
    except:
        # モデルを学習
        train_model_and_save()

## 29.6 Flaskアプリケーションの実装

In [ ]:
# Webアプリケーションのルート
@app.route('/')
def index():
    """トップページ"""
    return render_template('index.html')

@app.route('/upload', methods=['POST'])
def upload_file():
    """ファイルアップロードと予測"""
    if 'file' not in request.files:
        return jsonify({'error': 'ファイルがありません'}), 400
    
    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'ファイルが選択されていません'}), 400
    
    if file and allowed_file(file.filename):
        start_time = time.time()
        
        # 一時ファイルに保存
        with tempfile.NamedTemporaryFile(delete=False, suffix='.png') as tmp:
            tmp_path = tmp.name
            file.save(tmp_path)
        
        try:
            # 画像処理と予測
            image = feature_extractor.load_image(tmp_path)
            processed_image = feature_extractor.preprocess(image)
            features = feature_extractor.extract_features_combined(processed_image)
            
            # スケーリング
            features_scaled = scaler.transform([features])
            
            # 予測
            prediction = model.predict(features_scaled)[0]
            probabilities = model.predict_proba(features_scaled)[0]
            confidence = np.max(probabilities)
            
            # 画像をBase64エンコードして保存
            _, buffer = cv2.imencode('.png', processed_image)
            image_b64 = base64.b64encode(buffer).decode()
            
            # 処理時間
            processing_time = time.time() - start_time
            
            # 結果の保存
            filename = file.filename
            db.save_prediction(
                filename, 
                str(prediction), 
                float(confidence),
                processing_time=processing_time,
                image_data=image_b64
            )
            
            # 履歴に追加
            history.append({
                'filename': filename,
                'prediction': prediction,
                'confidence': confidence,
                'time': processing_time
            })
            
            # レスポンス
            response = {
                'success': True,
                'prediction': int(prediction),
                'confidence': float(confidence),
                'class_name': class_names[prediction],
                'processing_time': processing_time,
                'image': image_b64,
                'probabilities': {
                    class_names[i]: float(prob) 
                    for i, prob in enumerate(probabilities)
                }
            }
            
            return jsonify(response)
        
        finally:
            # 一時ファイルの削除
            os.unlink(tmp_path)
    
    return jsonify({'error': '許可されていないファイル形式です'}), 400

@app.route('/history')
def history_page():
    """履歴ページ"""
    return render_template('history.html')

@app.route('/api/history')
def api_history():
    """履歴API"""
    predictions = db.get_recent_predictions(20)
    return jsonify(predictions)

@app.route('/api/stats')
def api_stats():
    """統計情報API"""
    stats = db.get_statistics()
    return jsonify(stats)

@app.route('/api/recent')
def api_recent():
    """最近の予測API"""
    recent = list(history)
    return jsonify(recent)

def allowed_file(filename):
    """許可されたファイル形式かチェック"""
    ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif'}
    return '.' in filename and \
        filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

---

# Part 3: Web Templates

## 29.7 HTML/CSSテンプレート

In [ ]:
# templatesディレクトリの作成
os.makedirs('templates', exist_ok=True)

# index.htmlの作成
index_html = '''<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>画像分類アプリ</title>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
        }
        .container {
            max-width: 1200px;
            margin: 0 auto;
            background: white;
            border-radius: 10px;
            box-shadow: 0 10px 30px rgba(0,0,0,0.2);
            overflow: hidden;
        }
        .header {
            background: #2c3e50;
            color: white;
            padding: 30px;
            text-align: center;
        }
        .header h1 {
            font-size: 2.5rem;
            margin-bottom: 10px;
        }
        .header p {
            font-size: 1.1rem;
            opacity: 0.9;
        }
        .main {
            padding: 40px;
        }
        .upload-area {
            border: 3px dashed #3498db;
            border-radius: 10px;
            padding: 40px;
            text-align: center;
            transition: all 0.3s ease;
            cursor: pointer;
            margin-bottom: 30px;
        }
        .upload-area:hover {
            background: #ecf0f1;
            border-color: #2980b9;
        }
        .upload-area.dragover {
            background: #e3f2fd;
            border-color: #2196f3;
        }
        .upload-icon {
            font-size: 4rem;
            color: #3498db;
            margin-bottom: 20px;
        }
        .upload-text {
            font-size: 1.2rem;
            color: #2c3e50;
            margin-bottom: 20px;
        }
        .file-input {
            display: none;
        }
        .upload-btn {
            background: #3498db;
            color: white;
            border: none;
            padding: 12px 30px;
            border-radius: 25px;
            font-size: 1rem;
            cursor: pointer;
            transition: all 0.3s ease;
        }
        .upload-btn:hover {
            background: #2980b9;
            transform: translateY(-2px);
        }
        .result-container {
            display: none;
            margin-top: 30px;
        }
        .result-box {
            background: #f8f9fa;
            border-radius: 10px;
            padding: 30px;
            display: grid;
            grid-template-columns: 1fr 1fr;
            gap: 30px;
        }
        .image-preview {
            text-align: center;
        }
        .preview-img {
            max-width: 100%;
            max-height: 300px;
            border-radius: 10px;
            box-shadow: 0 5px 15px rgba(0,0,0,0.1);
        }
        .prediction-info {
            display: flex;
            flex-direction: column;
            justify-content: center;
        }
        .prediction-label {
            font-size: 1.5rem;
            color: #2c3e50;
            margin-bottom: 20px;
        }
        .prediction-value {
            font-size: 3rem;
            font-weight: bold;
            color: #3498db;
            margin-bottom: 20px;
        }
        .confidence {
            background: #e3f2fd;
            border-radius: 20px;
            padding: 15px;
            margin-bottom: 20px;
        }
        .confidence-label {
            font-size: 0.9rem;
            color: #666;
            margin-bottom: 5px;
        }
        .confidence-bar {
            background: #ddd;
            height: 20px;
            border-radius: 10px;
            overflow: hidden;
        }
        .confidence-fill {
            height: 100%;
            background: linear-gradient(90deg, #4caf50, #8bc34a);
            transition: width 0.5s ease;
        }
        .probabilities {
            margin-top: 20px;
        }
        .prob-item {
            display: flex;
            align-items: center;
            margin-bottom: 10px;
        }
        .prob-label {
            width: 30px;
            font-weight: bold;
            color: #2c3e50;
        }
        .prob-bar {
            flex: 1;
            background: #ecf0f1;
            height: 15px;
            border-radius: 10px;
            margin: 0 10px;
            overflow: hidden;
        }
        .prob-fill {
            height: 100%;
            background: #3498db;
            transition: width 0.5s ease;
        }
        .prob-value {
            width: 50px;
            text-align: right;
            font-size: 0.9rem;
            color: #666;
        }
        .loading {
            display: none;
            text-align: center;
            margin: 20px 0;
        }
        .spinner {
            border: 5px solid #f3f3f3;
            border-top: 5px solid #3498db;
            border-radius: 50%;
            width: 50px;
            height: 50px;
            animation: spin 1s linear infinite;
            margin: 0 auto;
        }
        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }
        .nav-links {
            display: flex;
            justify-content: center;
            gap: 20px;
            margin-top: 30px;
        }
        .nav-link {
            background: #3498db;
            color: white;
            padding: 10px 20px;
            border-radius: 5px;
            text-decoration: none;
            transition: all 0.3s ease;
        }
        .nav-link:hover {
            background: #2980b9;
            transform: translateY(-2px);
        }
        .stats {
            background: #f8f9fa;
            border-radius: 10px;
            padding: 20px;
            margin-bottom: 30px;
            text-align: center;
        }
        .stats-grid {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 20px;
            margin-top: 20px;
        }
        .stat-item {
            background: white;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
        }
        .stat-value {
            font-size: 2rem;
            font-weight: bold;
            color: #3498db;
        }
        .stat-label {
            color: #666;
            margin-top: 5px;
        }
        .error {
            background: #ffebee;
            border: 1px solid #f44336;
            color: #c62828;
            padding: 15px;
            border-radius: 5px;
            margin: 20px 0;
            display: none;
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>🔍 画像分類アプリ</h1>
            <p>手書き数字をAIが認識します</p>
        </div>
        
        <div class="main">
            <div class="stats" id="stats">
                <h3>統計情報</h3>
                <div class="stats-grid">
                    <div class="stat-item">
                        <div class="stat-value" id="total-predictions">0</div>
                        <div class="stat-label">総予測数</div>
                    </div>
                    <div class="stat-item">
                        <div class="stat-value" id="accuracy">0%</div>
                        <div class="stat-label">正解率</div
                    </div>
                    <div class="stat-item">
                        <div class="stat-value" id="avg-confidence">0%</div>
                        <div class="stat-label">平均信頼度</div>
                    </div>
                </div>
            </div>
            
            <div class="upload-area" id="uploadArea">
                <div class="upload-icon">📁</div>
                <div class="upload-text">ここに画像をドラッグ＆ドロップするか、クリックして選択</div>
                <button class="upload-btn" onclick="document.getElementById('fileInput').click()">ファイルを選択</button>
                <input type="file" id="fileInput" class="file-input" accept="image/*">
            </div>
            
            <div class="error" id="error"></div>
            
            <div class="loading" id="loading">
                <div class="spinner"></div>
                <p>画像を分析中...</p>
            </div>
            
            <div class="result-container" id="resultContainer">
                <div class="result-box">
                    <div class="image-preview">
                        <h3>入力画像</h3>
                        <img id="previewImage" class="preview-img" alt="Preview">
                    </div>
                    <div class="prediction-info">
                        <div class="prediction-label">予測結果</div>
                        <div class="prediction-value" id="predictionValue">-</div>
                        <div class="confidence">
                            <div class="confidence-label">信頼度</div>
                            <div class="confidence-bar">
                                <div class="confidence-fill" id="confidenceFill"></div>
                            </div>
                            <div id="confidenceText">0%</div>
                        </div>
                        <div class="probabilities" id="probabilities">
                            <h3>各クラスの確率</h3>
                        </div>
                    </div>
                </div>
            </div>
            
            <div class="nav-links">
                <a href="/" class="nav-link">ホーム</a>
                <a href="/history" class="nav-link">履歴</a>
            </div>
        </div>
    </div>
    
    <script>
        // ファイルアップロードの処理
        const uploadArea = document.getElementById('uploadArea');
        const fileInput = document.getElementById('fileInput');
        const loading = document.getElementById('loading');
        const resultContainer = document.getElementById('resultContainer');
        const errorDiv = document.getElementById('error');
        
        // ドラッグ＆ドロップ
        uploadArea.addEventListener('dragover', (e) => {
            e.preventDefault();
            uploadArea.classList.add('dragover');
        });
        
        uploadArea.addEventListener('dragleave', () => {
            uploadArea.classList.remove('dragover');
        });
        
        uploadArea.addEventListener('drop', (e) => {
            e.preventDefault();
            uploadArea.classList.remove('dragover');
            const files = e.dataTransfer.files;
            if (files.length > 0) {
                handleFile(files[0]);
            }
        });
        
        // ファイル選択
        fileInput.addEventListener('change', (e) => {
            if (e.target.files.length > 0) {
                handleFile(e.target.files[0]);
            }
        });
        
        function handleFile(file) {
            // 画像のプレビュー
            const reader = new FileReader();
            reader.onload = (e) => {
                document.getElementById('previewImage').src = e.target.result;
            };
            reader.readAsDataURL(file);
            
            // 送信
            const formData = new FormData();
            formData.append('file', file);
            
            loading.style.display = 'block';
            resultContainer.style.display = 'none';
            errorDiv.style.display = 'none';
            
            fetch('/upload', {
                method: 'POST',
                body: formData
            })
            .then(response => response.json())
            .then(data => {
                loading.style.display = 'none';
                
                if (data.success) {
                    // 結果の表示
                    document.getElementById('predictionValue').textContent = data.prediction;
                    document.getElementById('confidenceFill').style.width = (data.confidence * 100) + '%';
                    document.getElementById('confidenceText').textContent = (data.confidence * 100).toFixed(1) + '%';
                    
                    // 各クラスの確率
                    const probsContainer = document.getElementById('probabilities');
                    probsContainer.innerHTML = '<h3>各クラスの確率</h3>';
                    
                    Object.entries(data.probabilities).forEach(([digit, prob]) => {
                        const probItem = document.createElement('div');
                        probItem.className = 'prob-item';
                        probItem.innerHTML = `
                            <div class="prob-label">${digit}</div>
                            <div class="prob-bar">
                                <div class="prob-fill" style="width: ${prob * 100}%"></div>
                            </div>
                            <div class="prob-value">${(prob * 100).toFixed(1)}%</div>
                        `;
                        probsContainer.appendChild(probItem);
                    });
                    
                    resultContainer.style.display = 'block';
                    updateStats();
                } else {
                    errorDiv.textContent = data.error || 'エラーが発生しました';
                    errorDiv.style.display = 'block';
                }
            })
            .catch(error => {
                loading.style.display = 'none';
                errorDiv.textContent = '通信エラーが発生しました';
                errorDiv.style.display = 'block';
                console.error('Error:', error);
            });
        }
        
        // 統計情報の更新
        function updateStats() {
            fetch('/api/stats')
            .then(response => response.json())
            .then(data => {
                document.getElementById('total-predictions').textContent = data.total || 0;
                document.getElementById('accuracy').textContent = ((data.accuracy || 0) * 100).toFixed(1) + '%';
                document.getElementById('avg-confidence').textContent = ((data.avg_confidence || 0) * 100).toFixed(1) + '%';
            })
            .catch(error => console.error('Error updating stats:', error));
        }
        
        // ページ読み込み時に統計情報を更新
        window.addEventListener('load', updateStats);
    </script>
</body>
</html>
'''

with open('templates/index.html', 'w', encoding='utf-8') as f:
    f.write(index_html)

# history.htmlの作成
history_html = '''<!DOCTYPE html>
<html lang="ja">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>予測履歴 - 画像分類アプリ</title>
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
            margin: 0;
        }
        .container {
            max-width: 1200px;
            margin: 0 auto;
            background: white;
            border-radius: 10px;
            box-shadow: 0 10px 30px rgba(0,0,0,0.2);
            overflow: hidden;
        }
        .header {
            background: #2c3e50;
            color: white;
            padding: 30px;
            text-align: center;
        }
        .header h1 {
            font-size: 2rem;
            margin-bottom: 10px;
        }
        .main {
            padding: 40px;
        }
        .nav-links {
            display: flex;
            justify-content: center;
            gap: 20px;
            margin-bottom: 30px;
        }
        .nav-link {
            background: #3498db;
            color: white;
            padding: 10px 20px;
            border-radius: 5px;
            text-decoration: none;
            transition: all 0.3s ease;
        }
        .nav-link:hover {
            background: #2980b9;
            transform: translateY(-2px);
        }
        .history-table {
            width: 100%;
            border-collapse: collapse;
            margin-top: 20px;
            background: white;
            border-radius: 10px;
            overflow: hidden;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
        }
        .history-table th,
        .history-table td {
            padding: 15px;
            text-align: left;
            border-bottom: 1px solid #ecf0f1;
        }
        .history-table th {
            background: #34495e;
            color: white;
            font-weight: 600;
        }
        .history-table tr:hover {
            background: #f8f9fa;
        }
        .prediction-correct {
            color: #27ae60;
            font-weight: bold;
        }
        .prediction-incorrect {
            color: #e74c3c;
        }
        .confidence-bar {
            display: inline-block;
            width: 100px;
            height: 10px;
            background: #ecf0f1;
            border-radius: 5px;
            overflow: hidden;
            vertical-align: middle;
            margin-left: 10px;
        }
        .confidence-fill {
            height: 100%;
            background: #3498db;
        }
        .no-data {
            text-align: center;
            padding: 40px;
            color: #7f8c8d;
            font-size: 1.1rem;
        }
        .timestamp {
            font-size: 0.9rem;
            color: #7f8c8d;
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>📊 予測履歴</h1>
            <p>過去の分類結果を確認できます</p>
        </div>
        
        <div class="main">
            <div class="nav-links">
                <a href="/" class="nav-link">ホーム</a>
                <a href="/history" class="nav-link">履歴</a>
            </div>
            
            <h2>最近の予測</h2>
            <div id="history-container">
                <div class="no-data">読み込み中...</div>
            </div>
        </div>
    </div>
    
    <script>
        // 履歴の取得と表示
        function loadHistory() {
            fetch('/api/history')
            .then(response => response.json())
            .then(data => {
                const container = document.getElementById('history-container');
                
                if (data.length === 0) {
                    container.innerHTML = '<div class="no-data">予測履歴がありません</div>';
                    return;
                }
                
                let html = '<table class="history-table">';
                html += '<tr><th>日時</th><th>ファイル名</th><th>予測</th><th>実際</th><th>信頼度</th></tr>';
                
                data.forEach(item => {
                    const isCorrect = item.predicted_class === item.actual_class;
                    const correctClass = isCorrect ? 'prediction-correct' : 'prediction-incorrect';
                    const actualDisplay = item.actual_class ? item.actual_class : '-';
                    
                    html += `
                        <tr>
                            <td>${item.timestamp}</td>
                            <td>${item.filename}</td>
                            <td class="${correctClass}">${item.predicted_class}</td>
                            <td>${actualDisplay}</td>
                            <td>
                                ${item.confidence}
                                <div class="confidence-bar">
                                    <div class="confidence-fill" style="width: ${item.confidence * 100}%"></div>
                                </div>
                            </td>
                        </tr>
                    `;
                });
                
                html += '</table>';
                container.innerHTML = html;
            })
            .catch(error => {
                console.error('Error loading history:', error);
                document.getElementById('history-container').innerHTML = 
                    '<div class="no-data">履歴の読み込みに失敗しました</div>';
            });
        }
        
        // ページ読み込み時に履歴を表示
        window.addEventListener('load', loadHistory);
    </script>
</body>
</html>
'''

with open('templates/history.html', 'w', encoding='utf-8') as f:
    f.write(history_html)

---

# Part 4: Application Testing and Demo

In [ ]:
# テスト用の画像生成関数
def create_test_images(output_dir='static/test_images'):
    """テスト用の画像を生成"""
    os.makedirs(output_dir, exist_ok=True)
    
    # 0から9の数字の画像を生成
    for digit in range(10):
        # 32x32の画像を作成
        img = np.zeros((32, 32), dtype=np.uint8)
        
        # 数字を描画
        center = 16
        if digit == 0:
            cv2.circle(img, (center, center), 12, 200, 2)
        elif digit == 1:
            cv2.line(img, (center, 5), (center, 27), 200, 2)
        elif digit == 2:
            pts = np.array([[4, 27], [4, 16], [16, 4], [28, 16], [28, 27]], np.int32)
            cv2.polylines(img, [pts], False, 200, 2)
        elif digit == 3:
            cv2.ellipse(img, (center, 16), (10, 12), 0, 0, 180, 200, 2)
            cv2.ellipse(img, (center, 16), (10, 12), 0, 180, 360, 200, 2)
        elif digit == 4:
            cv2.line(img, (4, 4), (4, 16), 200, 2)
            cv2.line(img, (4, 16), (28, 16), 200, 2)
            cv2.line(img, (28, 4), (28, 28), 200, 2)
        elif digit == 5:
            pts = np.array([[4, 4], [28, 4], [28, 16], [4, 16], [4, 28], [28, 28]], np.int32)
            cv2.polylines(img, [pts], False, 200, 2)
        elif digit == 6:
            cv2.circle(img, (center, 16), 12, 200, 2)
            cv2.line(img, (center, 16), (center, 28), 200, 2)
        elif digit == 7:
            cv2.line(img, (4, 4), (28, 4), 200, 2)
            cv2.line(img, (28, 4), (16, 16), 200, 2)
            cv2.line(img, (16, 16), (16, 28), 200, 2)
        elif digit == 8:
            cv2.circle(img, (center, center), 12, 200, 2)
            cv2.line(img, (4, center), (28, center), 200, 2)
        elif digit == 9:
            cv2.circle(img, (center, center), 12, 200, 2)
            cv2.line(img, (center, center), (center, 4), 200, 2)
        
        # 保存
        filename = os.path.join(output_dir, f'digit_{digit}.png')
        cv2.imwrite(filename, img)
        print(f"生成: {filename}")

# テスト画像の生成
create_test_images()

print("テスト用画像の生成完了")
print("\nアプリケーションの起動方法:")
print("1. Flaskサーバーを起動: python app.py")
print("2. ブラウザで http://localhost:5000 にアクセス")
print("3. 画像をアップロードして分類を実行")

print("\n注意事項:")
print("- 実行にはFlaskとOpenCVが必要です")
print("- 最初の実行時にモデル学習が行われます（約30秒かかります）")
print("- 画像形式はPNG, JPG, JPEGがサポートされています")

# アプリケーションの起動用コード
if __name__ == '__main__':
    # テスト用のデモンストレーション
    print("=== 画像分類アプリデモンストレーション ===")
    
    # モデルがロードされているか確認
    if model is not None:
        print("モデルが正常にロードされました")
        
        # テストデータでの予測
        test_dir = 'static/test_images'
        if os.path.exists(test_dir):
            print("\nテスト画像での予測:")
            processor = ImageProcessor(target_size=(32, 32))
            
            correct = 0
            total = 0
            
            for digit in range(10):
                test_file = os.path.join(test_dir, f'digit_{digit}.png')
                if os.path.exists(test_file):
                    # 画像読み込み
                    img = processor.load_image(test_file)
                    processed = processor.preprocess(img)
                    features = processor.extract_features_combined(processed)
                    
                    # 予測
                    features_scaled = scaler.transform([features])
                    prediction = model.predict(features_scaled)[0]
                    confidence = np.max(model.predict_proba(features_scaled))
                    
                    print(f"Digit {digit}: 予測={prediction}, 信頼度={confidence:.3f}")
                    
                    if prediction == digit:
                        correct += 1
                    total += 1
            
            print(f"\nテスト精度: {correct/total:.2%}")
    else:
        print("モデルがロードされていません")
    
    print("\nWebアプリケーションを起動するには以下のコマンドを実行:")
    print("python app.py")
    print("その後、ブラウザで http://localhost:5000 にアクセス")

# app.pyの作成
app_code = '''# Flaskアプリケーション本体
from app import app
if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)
'''

with open('app.py', 'w') as f:
    f.write(app_code)

print("\napp.pyファイルを作成しました")

---

# Self-Check (理解度確認)

本日の学習内容を確認しましょう：

## プロジェクト設計
- [ ] Webアプリケーションのアーキテクチャを理解した
- [ ] システムの主要コンポーネントを把握した
- [ ] データフロー（画像入力から結果出力まで）を理解した

## 実装技術
- [ ] 画像処理パイプラインを実装した
- [ ] 複数の特徴量抽出方法を実装した
- [ ] Flaskアプリケーションを作成した
- [ ] データベース接続と操作を実装した

## モデル管理
- [ ] モデルの学習と保存を実装した
- [ ] 特徴量の正規化を実装した
- [ ] 信頼度評価を実装した

## Web技術
- [ ] REST APIを実装した
- [ ] HTML/CSS/JavaScriptのテンプレートを作成した
- [ ] ファイルアップロード機能を実装した
- [ ] レスポンス形式を設計した

## データベース
- [ ] SQLiteデータベースを設定した
- [ ] 履歴保存機能を実装した
- [ ] 統計情報取得を実装した

## 実用化
- [ ] デプロイ手順を理解した
- [ ] パフォーマンスを考慮した実装を行った
- [ ] エラーハンドリングを実装した

---

**お疲れ様でした！** Day 29はこれで終了です。

これで画像分類アプリケーションの開発が完了しました。最終的な成果物として：

1. 完全なWebアプリケーション（Flask + HTML/CSS/JS）
2. 画像処理パイプライン（前処理 + 特徴抽出）
3. 機械学習モデル（k-NNベースの分類器）
4. データベース（履歴管理）
5. テストデータセット

次回（Day 30）は、最終プロジェクトとしてこれまで学んだ知識を統合した総合アプリケーションを開発します。

復習課題：
1. アプリケーションを実際に動かしてみる
2. 新しい特徴量を追加して精度向上を試みる
3. モデルをより高度な分類器（例: SVM, Random Forest）に変更してみる
4. レスポンシブデザインでモバイル対応を追加してみる